# Convert Pytorch model to Onnx model and add Padding Operator at the beginning

In [10]:
import torch
import torch.nn as nn
import onnxruntime
import onnx
from PIL import Image
import numpy as np

In [11]:
onnxruntime.__version__

'1.8.1'

In [12]:
class UpConv7(nn.Module):
    def __init__(self):
        super(UpConv7, self).__init__()
        self.net = nn.Sequential(
            nn.ReplicationPad2d(7),
            nn.Conv2d(3, 16, 3, 1, 0),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(16, 32, 3, 1, 0),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(32, 64, 3, 1, 0),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(64, 128, 3, 1, 0),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(128, 128, 3, 1, 0),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(128, 256, 3, 1, 0),
            nn.LeakyReLU(0.1, inplace=True),
            nn.ConvTranspose2d(256, 3, 4, 2, 3)            
        )

    def forward(self, x):
        return self.net(x)

In [13]:
model = torch.load('wf_scale2x.pth')

In [14]:
dir(model)

['__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [15]:
type(model)

dict

In [16]:
model.keys()

dict_keys(['nunif_model', 'name', 'updated_at', 'kwargs', 'state_dict'])

In [17]:
model_state_dict = None
for key in model.keys():
    if key != 'state_dict':
        print(key, model[key])
    else:
        model_state_dict = model[key]

nunif_model 1
name waifu2x.upconv_7
updated_at 1540947281.5372806
kwargs {'in_channels': 3, 'out_channels': 3}


In [18]:
type(model_state_dict)

collections.OrderedDict

In [19]:
len(model_state_dict)

14

In [20]:
model_state_dict.keys()

odict_keys(['net.0.weight', 'net.0.bias', 'net.2.weight', 'net.2.bias', 'net.4.weight', 'net.4.bias', 'net.6.weight', 'net.6.bias', 'net.8.weight', 'net.8.bias', 'net.10.weight', 'net.10.bias', 'net.12.weight', 'net.12.bias'])

In [21]:
model_state_dict['net.0.bias']

tensor([ 0.1164,  0.1098, -0.1032,  0.1074, -0.2489, -0.0805, -0.1081,  0.0010,
         0.0022, -0.2050,  0.1065,  0.0624,  0.1108,  0.0573,  0.1068,  0.0162])

In [22]:
sr_model = UpConv7()

In [23]:
sr_model

UpConv7(
  (net): Sequential(
    (0): ReplicationPad2d((7, 7, 7, 7))
    (1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
    (2): LeakyReLU(negative_slope=0.1, inplace=True)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (4): LeakyReLU(negative_slope=0.1, inplace=True)
    (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (6): LeakyReLU(negative_slope=0.1, inplace=True)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (8): LeakyReLU(negative_slope=0.1, inplace=True)
    (9): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
    (10): LeakyReLU(negative_slope=0.1, inplace=True)
    (11): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
    (12): LeakyReLU(negative_slope=0.1, inplace=True)
    (13): ConvTranspose2d(256, 3, kernel_size=(4, 4), stride=(2, 2), padding=(3, 3))
  )
)

In [24]:
dir(sr_model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_name',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_version',
 'add_module',
 'apply',
 'bfloat16',
 'buffers',
 'children',
 'cpu',
 'cuda',
 'double',
 'dump_patches',
 'eval'

In [25]:
idx = 0
for par in sr_model.parameters():
    print(type(par), par.size())
    idx += 1
    if idx == 2:
        print(par)

<class 'torch.nn.parameter.Parameter'> torch.Size([16, 3, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([16])
Parameter containing:
tensor([ 0.1303,  0.1918, -0.0039, -0.0123,  0.0990, -0.1838, -0.0106, -0.1423,
         0.0463,  0.1413, -0.0250, -0.1327, -0.0404, -0.1180,  0.1072,  0.1724],
       requires_grad=True)
<class 'torch.nn.parameter.Parameter'> torch.Size([32, 16, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([32])
<class 'torch.nn.parameter.Parameter'> torch.Size([64, 32, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([64])
<class 'torch.nn.parameter.Parameter'> torch.Size([128, 64, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([128])
<class 'torch.nn.parameter.Parameter'> torch.Size([128, 128, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([128])
<class 'torch.nn.parameter.Parameter'> torch.Size([256, 128, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([256])
<class 'torch.nn.parameter.Parameter'> torch.Size([

In [26]:
sr_state_dict = sr_model.state_dict()

In [27]:
sr_state_dict.keys()

odict_keys(['net.1.weight', 'net.1.bias', 'net.3.weight', 'net.3.bias', 'net.5.weight', 'net.5.bias', 'net.7.weight', 'net.7.bias', 'net.9.weight', 'net.9.bias', 'net.11.weight', 'net.11.bias', 'net.13.weight', 'net.13.bias'])

In [28]:
with torch.no_grad():
    idx = 1
    i = 0
    for name, param in sr_model.named_parameters():        
        new_name = name.replace(str(idx), str(idx-1))
        print(name, new_name)
        param.copy_(model_state_dict[new_name])
        if i % 2 == 1:
            idx += 2
        i += 1

net.1.weight net.0.weight
net.1.bias net.0.bias
net.3.weight net.2.weight
net.3.bias net.2.bias
net.5.weight net.4.weight
net.5.bias net.4.bias
net.7.weight net.6.weight
net.7.bias net.6.bias
net.9.weight net.8.weight
net.9.bias net.8.bias
net.11.weight net.10.weight
net.11.bias net.10.bias
net.13.weight net.12.weight
net.13.bias net.12.bias


In [29]:
idx = 0
for par in sr_model.parameters():
    print(type(par), par.size())
    idx += 1
    if idx == 2:
        print(par)

<class 'torch.nn.parameter.Parameter'> torch.Size([16, 3, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([16])
Parameter containing:
tensor([ 0.1164,  0.1098, -0.1032,  0.1074, -0.2489, -0.0805, -0.1081,  0.0010,
         0.0022, -0.2050,  0.1065,  0.0624,  0.1108,  0.0573,  0.1068,  0.0162],
       requires_grad=True)
<class 'torch.nn.parameter.Parameter'> torch.Size([32, 16, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([32])
<class 'torch.nn.parameter.Parameter'> torch.Size([64, 32, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([64])
<class 'torch.nn.parameter.Parameter'> torch.Size([128, 64, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([128])
<class 'torch.nn.parameter.Parameter'> torch.Size([128, 128, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([128])
<class 'torch.nn.parameter.Parameter'> torch.Size([256, 128, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([256])
<class 'torch.nn.parameter.Parameter'> torch.Size([

In [30]:
new_onnx_model_name = 'wf_scale2x_new.pth'
torch.save(sr_model, new_onnx_model_name)

In [31]:
data = torch.rand(1, 3, 256, 256)
torch.onnx.export(sr_model, data, new_onnx_model_name, 
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input':{0:'batch_size', 2:'height', 3:'width'}, 'output': {0:'batch_size', 2:'height', 3:'width'}}
)